# Daily accumulation for non-UTC timezone for the ERA5-land accumulated variables

The following workflow demonstrates how to calculate the daily sum of the accumulate variables in ERA5-land data for a non-UTC time-zone.

The crux of this problem is that ERA5-land accumulates data from 00:00-UTC to 00:00-UTC. We must sample the data at the correct time steps, and then sum the relevant components to give an accumulation based on the time-zone of interest.

In this exmaple, we compute the daily accumulation of precipitation for a 10°x10° centres on Romania, where we want use a time zone of UTC+02:00.

In [1]:
# Import the libraries used in this notebook
import cdsapi
import xarray as xr
import pandas as pd

# Set the time zone shift as a variable so it is easy to change
TIME_ZONE_SHIFT_HOURS = +2

# Calculate the time the TIME-ZONE midnight in UTC
LOCAL_MIDNIGHT_IN_UTC = (0-TIME_ZONE_SHIFT_HOURS) % 24

## Download some raw hourly data

Here we choose the ERA5 land total_precipitation data for January 2024. As the data is partly accumulated in the archive, we just need to retreive the time-steps which effect our calculation, i.e. `00:00` and `00:00 + TIME_ZONE_SHIFT`.

We have chosen a coarse grid to reduce the size of the download for this demonstration.

In [3]:
TIME_STEPS = ['00:00', f"{LOCAL_MIDNIGHT_IN_UTC:02d}:00"]
client = cdsapi.Client() 
dataset = "reanalysis-era5-land"
request = {
    'product_type': ['reanalysis'],
    'variable': ['total_precipitation'],
    'date': '20240101/20240131',
    'time': TIME_STEPS,
    'area': [50, 20, 40, 30],
    'grid': [1, 1],
    'data_format': 'grib',
}
result_file = client.retrieve(dataset, request).download()

## Open the result file with xarray

Included here are the additional keyword arguments used in the backend by the CDS

In [4]:
ds = xr.open_dataset(
    result_file, time_dims=["valid_time"],
    coords_as_attributes=["surface", "number"],
)
ds

<xarray.Dataset> Size: 31kB
Dimensions:     (valid_time: 62, latitude: 11, longitude: 11)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 496B 2024-01-01 ... 2024-01-31T22...
  * latitude    (latitude) float64 88B 50.0 49.0 48.0 47.0 ... 42.0 41.0 40.0
  * longitude   (longitude) float64 88B 20.0 21.0 22.0 23.0 ... 28.0 29.0 30.0
Data variables:
    tp          (valid_time, latitude, longitude) float32 30kB ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-09-04T17:54 GRIB to CDM+CF via cfgrib-0.9.1...

## Compute the daily sum

The daily accumulation in the TIME-ZONE is the accumulation on the day of interest from `00:00 UTC` until `LOCAL_MIDNIGHT_IN_UTC`, added to the accumulation on day-1 from `LOCAL_MIDNIGHT_IN_UTC` until `24:00 UTC` (= `00:00 UTC` on the day of interest).



In [5]:
# Group the data by hour
ds_grouped_by_hour = ds.groupby("valid_time.hour")

# Then create new datasets for the UTC midnight and the local midnight
i_UTC_minight, i_local_midnight = ds_grouped_by_hour.groups
ds_UTC_midnight = ds.isel(valid_time=ds_grouped_by_hour.groups[i_UTC_minight])
ds_local_midnight = ds.isel(valid_time=ds_grouped_by_hour.groups[i_local_midnight])

ds_UTC_midnight

<xarray.Dataset> Size: 15kB
Dimensions:     (valid_time: 31, latitude: 11, longitude: 11)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 248B 2024-01-01 ... 2024-01-31
  * latitude    (latitude) float64 88B 50.0 49.0 48.0 47.0 ... 42.0 41.0 40.0
  * longitude   (longitude) float64 88B 20.0 21.0 22.0 23.0 ... 28.0 29.0 30.0
Data variables:
    tp          (valid_time, latitude, longitude) float32 15kB ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-09-04T17:54 GRIB to CDM+CF via cfgrib-0.9.1...

So that we can add and subtract the two datasets, we need to make sure that the coordinates are the same
We can do this by setting the coordinates of the local midnight dataset to the coordinates of the UTC midnight dataset,
but we must add one day as it must be joined to the day after the UTC midnight data:

In [10]:
ds_local_midnight = ds_local_midnight.assign_coords(
    valid_time=ds_UTC_midnight.valid_time + pd.Timedelta(days=1)
)
ds_local_midnight

<xarray.Dataset> Size: 15kB
Dimensions:     (latitude: 11, longitude: 11, valid_time: 31)
Coordinates:
  * latitude    (latitude) float64 88B 50.0 49.0 48.0 47.0 ... 42.0 41.0 40.0
  * longitude   (longitude) float64 88B 20.0 21.0 22.0 23.0 ... 28.0 29.0 30.0
  * valid_time  (valid_time) datetime64[ns] 248B 2024-01-02 ... 2024-02-01
Data variables:
    tp          (valid_time, latitude, longitude) float32 15kB ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-09-04T17:54 GRIB to CDM+CF via cfgrib-0.9.1...

We can then subtract the UTC midnight data from the local midnight data, and then shift back one day to so that the time-stamp is aligned with the UTC data.

In [14]:
# Subtract the UTC midnight data from the local midnight data
ds_local_to_utc_midnight = ds_UTC_midnight - ds_local_midnight
# Shift the time back one day
ds_local_to_utc_midnight = ds_local_to_utc_midnight.assign_coords(
    valid_time=ds_local_to_utc_midnight.valid_time - pd.Timedelta(days=1)
)
ds_local_to_utc_midnight

<xarray.Dataset> Size: 15kB
Dimensions:     (latitude: 11, longitude: 11, valid_time: 30)
Coordinates:
  * latitude    (latitude) float64 88B 50.0 49.0 48.0 47.0 ... 42.0 41.0 40.0
  * longitude   (longitude) float64 88B 20.0 21.0 22.0 23.0 ... 28.0 29.0 30.0
  * valid_time  (valid_time) datetime64[ns] 240B 2024-01-01 ... 2024-01-30
Data variables:
    tp          (valid_time, latitude, longitude) float32 15kB 2.027e-06 ... ...

We can then sum the two datasets to get the total precipitation for the day, in the local time zone.
 Xarray will drop any time steps where we don't have data for both the UTC and local midnight

In [13]:
ds_accum_local = ds_local_midnight + ds_local_to_utc_midnight
ds_accum_local

<xarray.Dataset> Size: 14kB
Dimensions:     (latitude: 11, longitude: 11, valid_time: 29)
Coordinates:
  * latitude    (latitude) float64 88B 50.0 49.0 48.0 47.0 ... 42.0 41.0 40.0
  * longitude   (longitude) float64 88B 20.0 21.0 22.0 23.0 ... 28.0 29.0 30.0
  * valid_time  (valid_time) datetime64[ns] 232B 2024-01-02 ... 2024-01-30
Data variables:
    tp          (valid_time, latitude, longitude) float32 14kB 0.01223 ... 0....

One final step is to consider whether the accumulation is for the previous day or the current day.
If the TIME_ZONE shift is negative, then the accumulation is for the previous day

In [9]:
shift = int(TIME_ZONE_SHIFT_HOURS < 0)
ds_accum_local = ds_accum_local.assign_coords(
    valid_time=ds_accum_local.valid_time + pd.Timedelta(days=shift)
)

ds_accum_local

<xarray.Dataset> Size: 14kB
Dimensions:     (latitude: 11, longitude: 11, valid_time: 29)
Coordinates:
  * latitude    (latitude) float64 88B 50.0 49.0 48.0 47.0 ... 42.0 41.0 40.0
  * longitude   (longitude) float64 88B 20.0 21.0 22.0 23.0 ... 28.0 29.0 30.0
  * valid_time  (valid_time) datetime64[ns] 232B 2024-01-02 ... 2024-01-30
Data variables:
    tp          (valid_time, latitude, longitude) float32 14kB 0.01223 ... 0....